<a href="https://colab.research.google.com/github/MrCepesh/Innopolis/blob/main/%D0%94%D0%97_13_(%D0%91%D0%BE%D0%BB%D1%8C%D1%88%D0%B8%D0%B5_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задание. Обучите модель классификации для цветков Iris'а**                

1.Взять данные iris.csv

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 43.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=2e2952639358eaacb745a1fa1a23b6fd2e9f375e1165961f2d47c9399d9e76f2
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, IntegerType, DateType
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession #to define a SparkSession

sc = SparkContext('local') #spark_connection
spark = SparkSession(sc) # open spark session

2. Преобразуем в pyspark

In [4]:
# Загрузим фаил с компьютера для работы в колаб
from google.colab import files
uploaded = files.upload()

Saving iris.csv to iris.csv


In [5]:
# Используем options() чтобы задать несколько опций:
# header=True - Для чтения использует первую строку в качестве имен столбцов
# sep="," - Устанавливает разделитель для каждого поля и значения
# quote="" - Нужно установить пустую строку для отключения кавычки, тк quote по-умолчанию ", тк ковычки не являются разделителем
df = spark.read.format("com.databricks.spark.csv").options(sep=",", header=True, quote="").csv('iris.csv')

In [6]:
df.show(10) # Узнаем какие данные храняться в фаиле
df.printSchema()

+-------------+---------------+----------------+---------------+------------+
|"sepal.length|""sepal.width""|""petal.length""|""petal.width""|""variety"""|
+-------------+---------------+----------------+---------------+------------+
|         "5.1|            3.5|             1.4|             .2| ""Setosa"""|
|         "4.9|              3|             1.4|             .2| ""Setosa"""|
|         "4.7|            3.2|             1.3|             .2| ""Setosa"""|
|         "4.6|            3.1|             1.5|             .2| ""Setosa"""|
|           "5|            3.6|             1.4|             .2| ""Setosa"""|
|         "5.4|            3.9|             1.7|             .4| ""Setosa"""|
|         "4.6|            3.4|             1.4|             .3| ""Setosa"""|
|           "5|            3.4|             1.5|             .2| ""Setosa"""|
|         "4.4|            2.9|             1.4|             .2| ""Setosa"""|
|         "4.9|            3.1|             1.5|             .1|

In [7]:
# преобразуем названия колонок
newColumns = ["sepal_length","sepal_width","petal_length","petal_width","variety"]
df_new = df.toDF(*newColumns)

# удалим лишние кавычки
df_new = df_new.withColumn("sepal_length", regexp_replace(col("sepal_length"), '"', ""))\
    .withColumn("variety", regexp_replace(col("variety"), '"', ""))

# преобразуем в тип данных Double
df_new = df_new \
    .withColumn('sepal_length', col('sepal_length').cast(DoubleType())) \
    .withColumn('sepal_width', col('sepal_width').cast(DoubleType())) \
    .withColumn('petal_length', col('petal_length').cast(DoubleType())) \
    .withColumn('petal_width', col('petal_width').cast(DoubleType()))

In [9]:
# посмотрим на итог преобразований
df_new.show(10)
df_new.printSchema()

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
|         5.4|        3.9|         1.7|        0.4| Setosa|
|         4.6|        3.4|         1.4|        0.3| Setosa|
|         5.0|        3.4|         1.5|        0.2| Setosa|
|         4.4|        2.9|         1.4|        0.2| Setosa|
|         4.9|        3.1|         1.5|        0.1| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 10 rows

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true

In [10]:
df_new.summary().show() # вычислим основные мат.значения наших данных

+-------+------------------+-------------------+------------------+------------------+---------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  variety|
+-------+------------------+-------------------+------------------+------------------+---------+
|  count|               150|                150|               150|               150|      150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     null|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     null|
|    min|               4.3|                2.0|               1.0|               0.1|   Setosa|
|    25%|               5.1|                2.8|               1.6|               0.3|     null|
|    50%|               5.8|                3.0|               4.3|               1.3|     null|
|    75%|               6.4|                3.3|               5.1|               1.8|     null|
|    max|               7.9|  

3.При помощи VectorAssemblerпреобразовать все колонки с признаками в одну (использовать PipeLine)

In [11]:
pipeline = Pipeline(stages = 
[StringIndexer(inputCol='variety', outputCol='varietyInd'), # переведем строковый формат в числовой для работы с данными
  VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol='Features')])

In [12]:
pipelineTrained = pipeline.fit(df_new) # подгоняем через заданный pipeline наш датасет
pipelineTrained.transform(df_new).show() # преобразуем признаки в вектор через функцию transform

+------------+-----------+------------+-----------+-------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|varietyInd|         Features|
+------------+-----------+------------+-----------+-------+----------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|       0.0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|       0.0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|       0.0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|       0.0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|       0.0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|       0.0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| Setosa|       0.0|[5.0,3.4,1.5,0.2]|
|         4.4|       

In [13]:
df_features = pipelineTrained.transform(df_new) # создаем новый датафрейм с признаком Features
df_features.show()

+------------+-----------+------------+-----------+-------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|varietyInd|         Features|
+------------+-----------+------------+-----------+-------+----------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|       0.0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|       0.0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|       0.0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|       0.0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|       0.0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|       0.0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| Setosa|       0.0|[5.0,3.4,1.5,0.2]|
|         4.4|       

4.Разбить данные на train и test

In [15]:
train, test = df_features.randomSplit([0.8, 0.2], seed=12345) # разделим датасет на обучающую (train) выборку и тестовую (test) в соотношениии 80/20
train.show()
test.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|
+------------+-----------+------------+-----------+----------+----------+-----------------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|       0.0|[4.3,3.0,1.1,0.1]|
|         4.4|        2.9|         1.4|        0.2|    Setosa|       0.0|[4.4,2.9,1.4,0.2]|
|         4.4|        3.0|         1.3|        0.2|    Setosa|       0.0|[4.4,3.0,1.3,0.2]|
|         4.4|        3.2|         1.3|        0.2|    Setosa|       0.0|[4.4,3.2,1.3,0.2]|
|         4.5|        2.3|         1.3|        0.3|    Setosa|       0.0|[4.5,2.3,1.3,0.3]|
|         4.6|        3.1|         1.5|        0.2|    Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         4.6|        3.4|         1.4|        0.3|    Setosa|       0.0|[4.6,3.4,1.4,0.3]|
|         4.6|        3.6|         1.0|        0.2|    Setosa|       0.0|[4.6,3.

5.Создать модель логистической регресии и обучить ее

In [16]:
lr = LogisticRegression(featuresCol = 'Features', labelCol = 'varietyInd') # возьмем признаки для обучается и признак для классифиции
lrModel = lr.fit(train)

In [17]:
# создадим предсказание через transform, которая принимает тестовую выборку, а также обучающую (как обучилась модель):
train_res = lrModel.transform(train)
test_res = lrModel.transform(test)
train_res.show() # для обучающей выборки

+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|       0.0|[4.3,3.0,1.1,0.1]|[67.7027919587531...|[1.0,1.6424718354...|       0.0|
|         4.4|        2.9|         1.4|        0.2|    Setosa|       0.0|[4.4,2.9,1.4,0.2]|[57.0087434717148...|[1.0,1.4036227809...|       0.0|
|         4.4|        3.0|         1.3|        0.2|    Setosa|       0.0|[4.4,3.0,1.3,0.2]|[61.7475610236625...|[1.0,2.4772179810...|       0.0|
|         4.4|        3.2|         1.3|        0.2|    Setosa|       0.0|[4.4,3.2,1.3,0.2]|[68.8060989728479...|[1.0,1.2455403381.

In [18]:
test_res.show() # для тестовой выборки

+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|         4.6|        3.2|         1.4|        0.2|    Setosa|       0.0|[4.6,3.2,1.4,0.2]|[65.8143591752207...|[1.0,8.9932431201...|       0.0|
|         5.0|        3.0|         1.6|        0.2|    Setosa|       0.0|[5.0,3.0,1.6,0.2]|[52.7723416307807...|[1.0,9.3248066295...|       0.0|
|         5.0|        3.2|         1.2|        0.2|    Setosa|       0.0|[5.0,3.2,1.2,0.2]|[64.6690738893863...|[1.0,1.7992914773...|       0.0|
|         5.0|        3.5|         1.3|        0.3|    Setosa|       0.0|[5.0,3.5,1.3,0.3]|[71.4022940655650...|[1.0,5.6805112138.

6.Воспользоваться MulticlassClassificationEvaluator для оценки качества на train и test множестве

In [20]:
ev = MulticlassClassificationEvaluator(labelCol='varietyInd') # MulticlassClassification применяем для более чем 2х признаков опираясь на varietyInd
print('Точность предсказания на обучающей выборке:', ev.evaluate(train_res)*100, '%') # проверим эффективность модели для обучающей выборки

Точность предсказания на обучающей выборке: 98.44961240310077 %


In [21]:
print('Точность предсказания на тестовой выборке:', ev.evaluate(test_res)*100, '%') # теперь проверим эффективность для тестовой выборки

Точность предсказания на тестовой выборке: 100.0 %


Вывод: 100 процентный результат для тестовой выборки показывает, что модель работает идеально.